single cell

In [ ]:
import scanpy as sc
from MGFEA.multiGraphFEA import GraphFEA, SpatialGraphFEA
from MGFEA import preprocess
import magic
import scipy as sp
from sklearn.metrics import mean_squared_error
import pandas as pd

adata = sc.read_h5ad('./data/dataset/h5ad/cortex_murine.h5ad')
##preprocess
sc.pp.highly_variable_genes(adata,n_top_genes=2000)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.filter_genes(adata,min_cells=1)

adata2 = magic.MAGIC().fit_transform(adata, genes='all_genes')
adata2.X = sp.sparse.csr_matrix(adata2.X)

##symbol is used to save gene symbol message
adata2.var['symbol'] = adata2.var['feature_name'].to_list()
graph = sc.read_h5ad('./data/gsmmmodel/scFEA_mouse.h5ad')


graph2 = preprocess.define_genesffea(adata2,graph,num_hvg=2000)
subset=adata2[:,adata2.var['genes4fea']]

fea_sc = GraphFEA(subset,graph2,train_epoch_num=100,batch_size=1000,loss_w = 0.5,lr=0.00001,save_lossfig='./model_loss.jpg')
#epoch 100 lr 0.0001
x,z = fea_sc.run()
met_data=pd.DataFrame(z.dot(graph2.X.todense().T))
met_data.columns=graph2.obs.index
metab_scale=(met_data-met_data.min(axis=0))/(met_data.max(axis=0)-met_data.min(axis=0))

spatial

In [ ]:
from MGFEA import preprocess
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import STAGATE
import tensorflow as tf
from MGFEA.multiGraphFEA import GraphFEA, SpatialGraphFEA

scdata = sc.read_h5ad('/ssd/metavelo_share/multiMetaFBA/data/gsmmmodel/IMM_mouse.h5ad')
cofactors=['oh1_c','fe3_c','h2o_c','o2_c','h_c','pi_c','hco3_c','na1_c','co2_c','so3_c','ppi_c','k_c','o2s_c','h2o2_c','nh4_c','k_c','ca2_c','cl_c',
'i_c','cyan_c','fe2_c','so4_c','tsul_c','zn2_c','fe2_c','tsul_c','fad_c','fadh2_c','nadp_c','nadph_c','nad_c','nadh_c','atp_c','amp_c','adp_c','coa_c']
scdata= scdata[~scdata.obs['id'].isin(cofactors),]

path_ref = '/ssd/metavelo_share/multiMetaFBA/data/dataset/spatial/sma-alldata/V11T16-085/V11T16-085_B1/output_data/V11T16-085_B1_RNA/outs'
sdata=sc.read_visium(path=path_ref,count_file="filtered_feature_bc_matrix.h5",source_image_path="spatial/")
sdata = sdata[:,~sdata.var.index.duplicated()]
sdata.var['symbol'] = sdata.var.index

sc.pp.normalize_total(sdata, inplace=True)
sc.pp.log1p(sdata)
sc.pp.highly_variable_genes(sdata, n_top_genes=500)
scdata2 = preprocess.define_genesffea(sdata,scdata,num_hvg=500)

sdata.var.index = sdata.var.symbol
sdata3 = sdata[:,sdata.var['genes4fea'].to_list()]

sdata3.var['highly_variable']=True

STAGATE.Cal_Spatial_Net(sdata3, rad_cutoff=1000)
STAGATE.Stats_Spatial_Net(sdata3)

tf.compat.v1.disable_eager_execution()
sdata3 = STAGATE.train_STAGATE(sdata3, alpha=0, save_reconstrction=True)
sdata3.X = sp.sparse.csr_matrix(sdata3.layers['STAGATE_ReX'])

spt_ver=sdata.obsm['spatial']
edge_index = preprocess.feature2adj(spt_ver,k=6)
edge_index = preprocess.adj2adjindex(edge_index)


fea_sc = SpatialGraphFEA(sdata3,edge_index,scdata2,pc_N=1024,lr=0.00001,train_epoch_num=100,loss_w=0.99,save_lossfig='./loss.jpg',coexp=True)
x, z = fea_sc.run()
m = z.dot(scdata2.X.todense().T)
flux = pd.DataFrame(z)
flux.columns = scdata2.var.index
flux.index = sdata3.obs.index
flux_t = flux.T

metb = pd.DataFrame(m)
metb.columns = scdata2.obs.index.to_list()
metb.index = sdata3.obs.index


reference

In [ ]:
from MGFEA import preprocess
from MGFEA.multiGraphFEA import RefGraphFEA

import scanpy as sc
import pandas as pd
import STAGATE
import tensorflow as tf

scdata = sc.read_h5ad('/ssd/metavelo_share/multiMetaFBA/data/gsmmmodel/IMM_mouse.h5ad')
cofactors=['oh1_c','fe3_c','h2o_c','o2_c','h_c','pi_c','hco3_c','na1_c','co2_c','so3_c','ppi_c','k_c','o2s_c','h2o2_c','nh4_c','k_c','ca2_c','cl_c',
'i_c','cyan_c','fe2_c','so4_c','tsul_c','zn2_c','fe2_c','tsul_c','fad_c','fadh2_c','nadp_c','nadph_c','nad_c','nadh_c','atp_c','amp_c','adp_c','coa_c']
scdata= scdata[~scdata.obs['id'].isin(cofactors),]

path_ref = '/ssd/metavelo_share/multiMetaFBA/data/dataset/spatial/sma-alldata/V11T16-085/V11T16-085_B1/output_data/V11T16-085_B1_RNA/outs'
sdata=sc.read_visium(path=path_ref,count_file="filtered_feature_bc_matrix.h5",source_image_path="spatial/")
sdata = sdata[:,~sdata.var.index.duplicated()]
sdata.var['symbol'] = sdata.var.index


msidata = sc.read_loom('/ssd/metavelo_share/multiMetaFBA/data/dataset/spatial/sma-alldata/msi/V11T16-085_B1_msi.loom')
msidata.obs['barcode'] = msidata.obs.index.str.rstrip('1').str.rstrip('_')
msidata.obs.index = msidata.obs['barcode']
peak_list = ['353.16818','444.20715','423.18201','435.20692','689.29822','690.27454']#'674.2805','444.20715',
metb_name = ['4abut_c','srtn_c','his__L_c','3moxtyr_c','34dhpac_c','nrpphr_c']#'dopa_c','srtn_c',
msidata2 = msidata[:,peak_list]
msidata2.var.index = metb_name
sc.pp.normalize_total(sdata, inplace=True)
sc.pp.log1p(sdata)
sc.pp.highly_variable_genes(sdata, n_top_genes=500)

sdata2 = sdata[msidata.obs.index,]
sdata = sdata2
scdata2 = preprocess.define_genesffea(sdata,scdata)
sdata.var.index = sdata.var.symbol
sdata3 = sdata[:,sdata.var['genes4fea'].to_list()]
sdata3.var['highly_variable']=True

STAGATE.Cal_Spatial_Net(sdata3, rad_cutoff=1000)
STAGATE.Stats_Spatial_Net(sdata3)

tf.compat.v1.disable_eager_execution()
sdata3 = STAGATE.train_STAGATE(sdata3, alpha=0, save_reconstrction=True)
sdata3.X = sp.sparse.csr_matrix(sdata3.layers['STAGATE_ReX'])
sdata3.obsm['maldi'] = (msidata2.to_df()-msidata2.to_df().mean(axis=0))/msidata2.to_df().std(axis=0)*0.1
#spt_ver=spatial.obs.loc[:,['X','Y']]
spt_ver=sdata.obsm['spatial']
edge_index = preprocess.feature2adj(spt_ver,k=6)
edge_index = preprocess.adj2adjindex(edge_index)

fea_sc = RefGraphFEA(sdata3,edge_index,scdata2,sdata3.obsm['maldi'],pc_N=1024,lr=0.0005,train_epoch_num=100,loss_w=0.19,loss_r = 0.8,coexp =True,
                            save_lossfig='loss.jpg')
x, z = fea_sc.run()
m = z.dot(scdata2.X.todense().T)
flux = pd.DataFrame(z)
flux.columns = scdata2.var.index
flux.index = sdata3.obs.index
flux_t = flux.T

metb = pd.DataFrame(m)
metb.columns = scdata2.obs.index.to_list()
metb.index = sdata3.obs.index